# Download SRA annotations

## Load libraries

In [1]:
suppressPackageStartupMessages(library(reutils))
suppressPackageStartupMessages(library(xml2))
suppressPackageStartupMessages(library(tidyverse))

## Define analysis-specific variables

In [2]:
bioproject_number <- "PRJNA785998"

## Download SRA annotations and process it into required columns

In [3]:
sra_annotations <- bioproject_number %>% 
    esearch(db = "sra") %>% 
    efetch(db = "sra") %>% 
    content(as = "text") %>% 
    read_xml() %>% 
    xml_find_all("//EXPERIMENT") %>% 
    map_df(xml_attrs) %>% 
    mutate(data = str_split(alias, "_", 2)) %>% 
    unnest_wider(data, names_sep = "_") %>% 
    rename(sample_id = data_1, sample_name = data_2) %>% 
    select(-alias) %>% 
    write_tsv("../../annotations/sra_annotations.tsv")

sra_annotations

accession,sample_id,sample_name
<chr>,<chr>,<chr>
SRX13310313,146p28,stall4control4_linkage
SRX13310312,146p27,stall4control4_linkage
SRX13310311,146p26,stall4control4_linkage
SRX13310310,146p25,stall4control4_linkage
SRX13310309,146p24,stall4control4_linkage
SRX13310308,146p23,stall4control4_linkage
SRX13310307,139p7,endo12k_linkage
SRX13310306,146p16,stall4control4_t8
SRX13310305,146p15,stall4control4_t4
